In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm
from collections import deque, defaultdict

In [55]:
input_text = au.read_txt_file_lines()
n_rows = len(input_text)
# n_cols = len(input_text[0])
# for ii in range(1, n_rows):
#     assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
# print(f'input has {n_rows} rows and {n_cols} cols')
n_rows

585

In [56]:
graph = defaultdict(list)

for l in input_text:
    inp, out = l.split(': ')
    out = out.split()
    graph[inp] = out 

# graph

In [74]:
queue = deque(['you'])
total = 0
while len(queue) > 0:
    node = queue.popleft() 
    if node == 'out':
        total += 1
        continue 
    for new_node in graph[node]:
        queue.append(new_node)

total

746

## part 2

Problem can be divided into n_paths from `svr` to `fft`, `fft` to `dac`, `dac` to `out` and with `fft`/`dac` swapped.

- NO paths from `dac` to `fft` (makes sense because it should be a DAC graph)

Same BFS approach is too slow and memory intensive .. but finding n paths is a typical BFS problem. However, we don't care about the previous path (if we solve it in 3 stages). So we just need to have number of paths that reached a node. So we can solve it with adjacency matrix + vector of counts

In [62]:
## Create adjacency matrix:

names = list(graph.keys())
if 'out' not in names:
    names.append('out')
names = np.array(names)

dict_map = {n: k for k, n in enumerate(names)}
adj = np.zeros((len(names), len(names)))

for n, outs in graph.items():
    ind_n = dict_map[n]
    for n_out in outs:
        ind_o = dict_map[n_out]
        adj[ind_o, ind_n] = 1

assert np.all(np.diag(adj) == 0)

In [63]:
def get_n_paths(start='you', end='out', names=names, adj=adj):
    id_start = np.where(names == start)[0]
    id_end = np.where(names == end)[0]
    assert len(id_start) == 1
    assert len(id_end) == 1

    c = np.zeros(len(names), dtype=int)  ## counts, start with 1 for start node
    c[id_start] = 1 
    max_iter = 1000
    sum_end = 0 
    for it in range(max_iter):
        new_c = adj @ c  ## graph step
        sum_end += new_c[id_end][0]  # add all nodes that have reached end. (because adj[k, k] = 0 for all k, all traversal will only be at end node for 1 step)
        diff = np.sum(new_c - c)
        if diff == 0:  ## no more changes, stop
            print('no diff')
            break 
        c = new_c
        if it == max_iter - 1:
            print('max iter')

    sum_end = int(sum_end)
    if sum_end == 0:
        print(f'no paths found from {start} to {end}')
    return sum_end

get_n_paths('svr', 'fft') * get_n_paths('fft', 'dac') * get_n_paths('dac', 'out') + get_n_paths('svr', 'dac') * get_n_paths('dac', 'fft') * get_n_paths('fft', 'out')

no diff
no diff
no diff
no diff
no diff
no paths found from dac to fft
no diff


370500293582760